### Generic header ...

Utility functions were reduced to libits.py and removed from this notebook

In [1]:
import numpy as np
import pandas as pd
import shapely
import matplotlib
from matplotlib import pylab as plt
import pygeos
import geopandas as gpd
import pyproj
import contextily as ctx
import networkx as nx
import osmnx
import scipy as sc
import sklearn
import overpy
import tempfile
import os

from shapely.geometry import Point
from shapely.geometry import LineString

In [ ]:
crs_gps = pyproj.CRS.from_epsg('4326')  # Merkator
crs_web = pyproj.CRS.from_epsg('3857')  # Pseudo-Merkator
crs_estonia = pyproj.CRS.from_epsg('3301')

In [ ]:
overpass_client = overpy.Overpass()

In [ ]:
query=\
'''[out:json];
area["ISO3166-1"="FI"]->.a;

way[power=line](area.a);
(._;>;);   // Resolve nodes
out geom;
'''

In [ ]:
overpass_r = overpass_client.query(query)

In [ ]:
df_nodes =\
pd.DataFrame([dict([(a,getattr(x,a)) for a in ['id','lat','lon']]) for x in overpass_r.nodes])\
    .set_index('id',drop=True)
df_nodes

In [ ]:
df_ways_idx =\
pd.DataFrame(np.vstack([np.asarray([[w.id, n.id] for n in w.nodes])
                        for w in overpass_r.ways]),
             columns=['id','node'])\
    .set_index('id', drop=True)
df_ways_idx

In [ ]:
df_ways=\
    df_ways_idx.groupby('id').apply(lambda df: df.node.values)\
        .to_frame()\
        .rename(columns={0:'nodes'})
df_ways

In [ ]:
gdf_nodes = gpd.GeoDataFrame(
    df_nodes.assign(geometry=\
        df_nodes.groupby('id')\
            .apply(lambda df: df[['lon','lat']].values.reshape(-1))\
            .apply(Point))\
        .drop(columns=['lon','lat']),
    crs=crs_gps)
gdf_nodes

gdf_nodes

In [ ]:
ax = gdf_nodes.to_crs(crs_web).plot()
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
gdf_ways = gpd.GeoDataFrame(
    df_ways.assign(geometry=\
        df_ways.nodes.apply(lambda nodes: LineString(df_nodes.loc[nodes][['lon','lat']].values))),
    crs=crs_gps)
gdf_ways

In [ ]:
def get_voltage(x):
    try:
        vol = x.tags['voltage']
        if ';' in vol:
            vol = vol.split(';')[0]
        return vol
    except:
        return 0

In [ ]:
def get_cables(x):
    try:
        num = x.tags['cables']
        if num == '2x2':
            num = 4
        return num
    except:
        return 1

In [ ]:
df_capacity = pd.DataFrame([dict([('voltage',get_voltage(x)), ('cable_num',get_cables(x)), ('id',x.id)]) for x in overpass_r.ways]).set_index('id', drop=True)

In [ ]:
gdf_ways['voltage'] = pd.to_numeric(df_capacity.voltage)
gdf_ways['cable_num'] = pd.to_numeric(df_capacity.cable_num)

In [ ]:
# assumed
max_amp = 700
gdf_ways['watts'] = gdf_ways.voltage * gdf_ways.cable_num * max_amp

In [ ]:
ax=gdf_ways.to_crs(crs=crs_web).plot()
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
power_plants = pd.read_csv('plants.csv')

In [ ]:
power_plants = power_plants.loc[power_plants.country == 'FIN']

In [ ]:
len(power_plants)

In [ ]:
power_plants['geometry'] = power_plants.apply(lambda plant: Point(plant.longitude, plant.latitude),axis=1)

In [ ]:
power_plants_gdf = gpd.GeoDataFrame(power_plants, crs=crs_gps, geometry='geometry')

In [ ]:
ax=power_plants_gdf.to_crs(crs_web).plot()
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
power_plants_gdf = gpd.sjoin_nearest(power_plants_gdf, gdf_nodes, distance_col='distance')

In [ ]:
power_plants_gdf['new_geometry'] = power_plants_gdf.apply(lambda plant: gdf_nodes.loc[plant.index_right],axis=1)

In [ ]:
gdf_power_plants_on_grid = gpd.GeoDataFrame(power_plants_gdf, geometry='new_geometry', crs=crs_gps)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,20))

gdf_ways.to_crs(crs=crs_web).plot(ax=ax)
gdf_power_plants_on_grid.to_crs(crs=crs_web).plot(ax=ax,color='orange')
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
# Break down to road segments
#    Way can contain >=2 nodes
#    Segment contains exactly 2 nodes (start, end)
# Index the segment of each way
def road2segs(df_road):
    return \
        pd.DataFrame(dict(
            zip(['power_line','seg','start_node','end_node', 'watts'],
                np.asarray(list(
                    zip((np.ones(len(df_road.nodes)-1)*df_road.name).astype(np.int64),
                         np.arange(len(df_road.nodes)-1),
                         df_road.nodes[:-1],
                         df_road.nodes[1:]))).T))).set_index(['power_line','seg'])

In [ ]:
df_edges =\
    pd.concat([road2segs(gdf_ways.loc[way]) for way in gdf_ways.index])

gdf_edges=\
    gpd.GeoDataFrame(
        df_edges.assign(geometry=\
            df_edges\
                .join(gdf_nodes, on='start_node')\
                .rename(columns={'geometry':'vtx1'})\
                .join(gdf_nodes, on='end_node')\
                .rename(columns={'geometry':'vtx2'})\
                [['vtx1','vtx2']].T.apply(LineString)),
        crs=crs_gps)

In [ ]:
gdf_edges = gdf_edges.join(gdf_ways['watts'], on='power_line')

In [ ]:
ax=gdf_edges.to_crs(crs=crs_web).plot()
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
gdf_power_plants_on_grid = gdf_power_plants_on_grid.rename(columns={'index_right':'node_id', 'geometry':'original_geometry', 'new_geometry':'geometry'})

In [ ]:
#######################################
gdf_nodes
gdf_edges
gdf_power_plants_on_grid
#######################################

In [ ]:
#traverse graph to kick out

In [ ]:
import networkx as nx
G = nx.Graph()

In [ ]:
G.add_nodes_from(gdf_nodes.index, location=gdf_nodes.geometry)

In [ ]:
gdf_edges['edge'] = gdf_edges.apply(lambda edge: (edge.start_node,edge.end_node),axis=1)


In [ ]:
gdf_edg

In [ ]:
gdf_edges.reset_index()
gdf_edges = gdf_edges.set_index('edge')

In [ ]:
gdf_edges = gdf_edges.drop(['start_node', 'end_node', 'geometry'],axis=1)

In [ ]:
import json

with open('grid_contracted_nonan.json', 'r') as infile1:
    F = nx.node_link_graph(json.loads(infile1.read()))
D = nx.DiGraph(F)
D.add_node(0,power=0) # source
D.add_node(1,power=0) # sink

for n in D:
    power = D.nodes[n]['power']
    if power > 0:
        # producers
        D.add_edge(0,n,capacity=power)
    if power < 0:     
        # consumers
        D.add_edge(n,1,capacity=abs(power))

In [ ]:
import json
with open('grid_contracted.json', 'w') as outfile1:
    outfile1.write(json.dumps(nx.node_link_data(D)))

In [ ]:
import sys
sys.setrecursionlimit(100000)

def dfs(visited,G_orig,u):
    visited.add(u)

    for neighbor in G_orig.neighbors(u):
        if (neighbor not in visited):
            if (G_orig.nodes[neighbor]['power'] == 0):
                dfs(visited,G_orig,neighbor)
            if (G_orig.nodes[neighbor]['power'] != 0):
                # other node to keep
                #print(start, neighbor)
                cap = G_orig.edges[u,neighbor]['capacity']
                create_new_edge(start,neighbor,cap)
        if (neighbor in visited) and (G_orig.nodes[neighbor]['power'] != 0) and (neighbor != start):
            cap = G_orig.edges[u,neighbor]['capacity']
            update_edge(start,neighbor,cap)

In [ ]:
def create_new_edge(u,v,cap):
    G.add_edge(u,v,capacity=cap)

def update_edge(u,v,cap):
    G[u][v]['capacity'] += cap    

In [ ]:
with open('grid.json', 'r') as infile1:
    G = nx.node_link_graph(json.loads(infile1.read()))
G_orig = nx.Graph(G)

for n in G_orig:
    if G_orig.nodes[n]['power'] != 0:    
        visited = set()
        start = n
        dfs(visited,G_orig,n)

In [ ]:
kick=[]
for n in G:
    if G.nodes[n]['power'] == 0:
        kick.append(n)
        
for n in kick:
    G.remove_node(n)

In [ ]:
len(list(G.nodes))

In [ ]:
b = nx.to_pandas_edgelist(G)
b

In [ ]:
a = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
a['geometry'] = a.apply(lambda node: Point(node.lon, node.lat),axis=1)
a = gpd.GeoDataFrame(a, geometry='geometry', crs=crs_gps)

In [ ]:
b['geo'] = b.apply(lambda path: LineString([a.loc[path.source]['geometry'], a.loc[path.target]['geometry']]), axis=1)

In [ ]:
b = gpd.GeoDataFrame(b, geometry='geo', crs=crs_gps)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,20))
b.to_crs(crs_web).plot(ax=ax)
ctx.add_basemap(ax=ax, zoom=9)

In [ ]:
b.to_csv('contracted_paths.csv')
a.to_csv('contracted_nodes.csv')

In [ ]:
import json
with open('grid_contracted.json', 'w') as outfile1:
    outfile1.write(json.dumps(nx.node_link_data(G)))

In [ ]:
import math
for n in G:
    if math.isnan(G.nodes[n]['power']):
        print("!")
        G.nodes[n]['power'] = 0

In [ ]:
with open('grid_contracted_nonan.json', 'w') as outfile1:
    outfile1.write(json.dumps(nx.node_link_data(G)))

In [ ]:
water = osmnx.geometries_from_place('Finland', tags={'natural':'water'})

In [ ]:
house = osmnx.geometries_from_place('Finland', tags={'building':True})

In [ ]:
national = osmnx.geometries_from_place('Finland', tags={'boundary':'national_park'})

In [ ]:
parks = national.loc['relation']


In [ ]:
gdf_parks = gpd.GeoDataFrame(parks, geometry='geometry', crs=crs_gps)

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely.geometry import Point, LineString, Polygon, LinearRing, GeometryCollection

poly_parks = unary_union(GeometryCollection(list(gdf_parks.geometry)))
poly_parks

In [ ]:
lines = pd.read_csv('contracted_paths.csv')
lines = gpd.GeoDataFrame(lines, geometry='geo', crs=crs_gps)

In [ ]:
from shapely.ops import split
from shapely.wkt import loads

lines['split'] = lines.apply(lambda row: split(loads(row.geo), poly_parks), axis=1)

In [ ]:
lines['park'] = lines.apply(lambda row: loads(row.geo).intersects(poly_parks), axis=1)

In [ ]:
lines[lines['park']]

In [ ]:
lines['split']

In [ ]:
lines.info()

In [ ]:
with open('grid_contracted.json', 'r') as infile1:
    F = nx.node_link_graph(json.loads(infile1.read()))

In [ ]:
for n in F:
    F.nodes[n]['power'] = F.nodes[n]['power'] * (365*24)

In [ ]:
for s,e in F.edges:
    F[s][e]['capacity'] = F[s][e]['capacity'] / 1000000000

In [ ]:
for n in F:
    power = F.nodes[n]['power']
    if power > 0:
        # producers
        F[0][n]['capacity']=power
    if power < 0:     
        # consumers
        F[n][1]['capacity']=abs(power)

In [ ]:
F.edges.data()

In [ ]:
with open('grid_contracted_units.json', 'w') as outfile1:
    outfile1.write(json.dumps(nx.node_link_data(F)))

In [ ]:
lines

In [2]:
cuts = [(1147151748, 3536595671), (824065452, 4447358451), (634989945, 2636358023), (2408331411, 2408331526), (2647668116, 1568817948), (2156877608, 4011659942), (2278030269, 2759244272), (7833850369, 7833827236), (246529612, 294854486), (6175259891, 288465276), (1092860862, 5048354290), (1224394666, 1425122020), (1802301146, 5931192731), (2408331345, 2408331223), (2376430694, 1019905622), (1224394666, 2090571394), (8661578399, 8661598578), (3943008213, 3943008240)]

In [ ]:
lines = lines.set_index(['source','target'])

In [ ]:
selection = []
for a,b in cuts:
    try :
        selection.append([a,b,lines.loc[a,b].park])
    except:
        pass

In [12]:
pd.DataFrame(cuts).rename(columns={0:'source',1:'target'}).T.to_json('cuts.json')